In [2]:
import os
import numpy as np
from scipy import stats
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
sns.set(rc={'figure.figsize':(12,8)})


# Venue Network Analysis

Understanding why Treatwell users choose particular salons is crucial to optimize different aspects of our web appearance. There are many factors a user could consider before booking with a particular salon: treatment type, price, location and the salon's rating are just a few examples. The question is, which of all these factors are the most important ones.

A possible approach to answer this question is looking at the venues' customer circles. We can assume that a user always has the same priorities when picking a new salon. Hence salons with overlapping customer circles should share these attributes that are so important to their clienteles. Accordingly, clustering venues with similar customs and checking for shared attributes will give us an idea about what made these customers choose these salons.

This notebook builds the network of Treatwell's salons in London, using historic venue and order data. The table can then be loaded into Gephi, which allows visualizing and exploring the network.

Read my [article on Medium](link) to get a full overview of the project.


In [3]:
# loading order data
venue_df = pd.read_csv('venues_df.csv')
venue_df.head(5)

,Unnamed: 0,city,customer_account_id,has_been_top_rated_venue,venue_details_key,venue_name,payment_amount,order_count
0,0,West London,Eoquwse,Yes,Yg7xf4,Pcikquh9xy28gkhg0,90.00,6
1,1,Eastern Central London,Bckqcrk,Yes,U5lhvf,8barqoew2ri8jb9v1u0hqems2l,288.00,10
2,2,South West London,N3jxd3n,No,Yqxbad,Ptg74raeuxqpwvs6vxe0rq7dl,280.00,10
3,3,Kingston upon Thames,Pm22gng,Yes,Vefzhj,3pkwo9ms8z2yedoxuk,78.80,2
4,4,West London,Dmuzpne,Yes,Ma1vuj,Fcfhcrhklkoso1qwqithywms2l,251.01,11


In [ ]:
# loading venue treatment type data 
venue_tt_df = pd.read_csv('venues_tt_df.csv')
venue_tt_df.head(5)

In [4]:
print(len(venue_df))
# print(venue_df.head(5))

users_per_venue = venue_df[['venue_details_key', 'customer_account_id']] \
    .groupby(['venue_details_key'], as_index=False) \
    .count()

nr = list(users_per_venue['customer_account_id'])
perc = np.percentile(nr, 33)
print(perc)

users_per_venue = users_per_venue[users_per_venue['customer_account_id'] >= 50]
users_per_venue.rename(columns={'customer_account_id':'customer_count'}, inplace=True)

venue_df_small = venue_df.merge(users_per_venue, on='venue_details_key', how='inner')
venue_df_small.head()

253982
7.0


,Unnamed: 0,city,customer_account_id,has_been_top_rated_venue,venue_details_key,venue_name,payment_amount,order_count,customer_count
0,0,West London,Eoquwse,Yes,Yg7xf4,Pcikquh9xy28gkhg0,90.0,6,157
1,2845,West London,Woukxzn,Yes,Yg7xf4,Pcikquh9xy28gkhg0,287.8,8,157
2,3065,West London,Fjxo0v,Yes,Yg7xf4,Pcikquh9xy28gkhg0,243.9,7,157
3,7595,West London,Lsnxc9n,Yes,Yg7xf4,Pcikquh9xy28gkhg0,335.0,7,157
4,8866,West London,Ac pvrw,Yes,Yg7xf4,Pcikquh9xy28gkhg0,114.1,3,157


To identify salons with similar customer circles, an eligible metric is needed to measure the similarity between two sets. A popular and straight forward measure for cases like this is the Jaccard Coefficient.

Unfortunately, the index has a critical drawback since it ranks connections between two entities with an equal degree higher. Imagine the situation in the figure below, with a very big salon (S1) whose customer circle overlaps with many other salons and a very small salon (S2) whose clientele overlaps with the one of S1, but to a very high degree. Obviously we wanted this connection to be ranked very high. But since S1 shares customers with so many other venues, the score would be fairly low.

Therefore, I defined an alternative metric to measure the similarity between to venues. Using this metric, the similarity between S1 and S2, where the connection is only meaningful for one of the parties, would be around 0.5. If the connection is important for both sides, the value converges to 1 while it approximates 0 for very weak relations on both sides.

In [8]:
def jaccard_similarity(list1, list2):
    """
    Calculates jaccard similarity for two lists.
    """
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection / union)

def similarity_two_sided(list1, list2):
    """
    Alternative approach to calculate similarity of two lists.
    """
    intersection = len(list(set(list1).intersection(list2)))
    union_1 = float(intersection / len(list1))
    union_2 = float(intersection / len(list2))
    return (union_1 + union_2)/2

venue_ids = set(venue_df_small['venue_details_key'])

venue_connections = []
right_venue_list = []
left_venue_list = []

# build dictionary containing the customers for every salon
venues_with_customers = dict()
for venue_id in venue_ids:
    venue_customers = set(venue_df_small[venue_df_small['venue_details_key'] == venue_id]['customer_account_id'])
    venues_with_customers[venue_id] = venue_customers

# calculate similarity for every venue combination
used = []
for left_venue_id in venue_ids:
    for right_venue_id in venue_ids:
        if(right_venue_id) not in used:
            similarity = similarity_two_sided(venues_with_customers[left_venue_id], venues_with_customers[right_venue_id])
            venue_connections += [similarity]
            right_venue_list += [right_venue_id]
            left_venue_list += [left_venue_id]
    used += [left_venue_id]

# combine the three lists in a dataframe (ultimately the network table)
similarity_df = pd.DataFrame()
similarity_df['source'] = left_venue_list
similarity_df['target'] = right_venue_list
similarity_df['weight'] = venue_connections
similarity_df.head()


In [ ]:
# add venue names, drop irrelevant rows
names_ids = venue_df_small[['venue_details_key', 'venue_name']] \
    .rename(columns={'venue_name': 'source_name', 'venue_details_key': 'source'}) \
    .drop_duplicates()
similarity_df= similarity_df.merge(names_ids, on='source', how='inner')

names_ids = venue_df_small[['venue_details_key', 'venue_name']] \
    .rename(columns={'venue_name': 'target_name', 'venue_details_key': 'target'}) \
    .drop_duplicates()

similarity_df= similarity_df.merge(names_ids, on='target', how='inner')
similarity_df = similarity_df[(similarity_df['source'] != similarity_df['target']) \
                              & (similarity_df['weight'] > 0)]
similarity_df.head()

In [ ]:
# similarity_df.sort_values('weight', ascending=False)

In [ ]:
# keep only most important 20% edges for each node
similarity_df['rank'] = similarity_df.sort_values(['weight'], ascending=[False]) \
             .groupby(['source']) \
             .cumcount() + 1
similarity_df['count'] = similarity_df.groupby(['source'])['source'].transform('count') 
print(len(similarity_df[similarity_df['count'] >= 5]))
similarity_df['count'] = similarity_df['count'] * 0.2
similarity_df = similarity_df[similarity_df['rank'] <= similarity_df['count']]
print(len(similarity_df))


In [ ]:
# saving the edges table
similarity_df['weight'] = similarity_df['weight'] * 10
similarity_df.to_csv('/Users/dennismeisner/Documents/Venue_Network/jaccard_2.csv', index=False)

In [9]:
# creating a separate table with information about the network's nodes 
# (basically salon information like name, top_rated, area, average order value etc.)
nodes = venue_df_small[['city', 'venue_details_key', 'venue_name', 'has_been_top_rated_venue', \
                        'order_count', 'payment_amount']]
nodes = nodes.merge(venue_tt_df, on='venue_details_key', how='inner') 
nodes = nodes.rename(columns={'venue_details_key': 'id', 'venue_name':'label'})
nodes = nodes.groupby(['city', 'id', 'label', 'has_been_top_rated_venue', 'treatment_type'], as_index=False) \
    .sum()
nodes['avg_order_val'] = ((nodes['payment_amount'] / nodes['order_count']) /5).astype('int') * 5
nodes['order_count'] = ((nodes['order_count']) /5).astype('int') * 5
nodes = nodes[['city', 'id', 'label', 'has_been_top_rated_venue', 'treatment_type', 'order_count', 'avg_order_val']]
print(nodes.head())
nodes.to_csv('/Users/dennismeisner/Documents/Venue_Network/nodes_2.csv', index=False)

NameError: name 'venue_tt_df' is not defined

In [11]:
list_1 = ['salon_1', 'salon_1', 'salon_2']
list_2 = ['salon_2', 'salon_3', 'salon_3']
list_3 = [0.0, 0.45, 0.33]

k = pd.DataFrame()
k['source'] = list_1
k['target'] = list_2
k['weight'] = list_3
k

,source,target,weight
0,salon_1,salon_2,0.00
1,salon_1,salon_3,0.45
2,salon_2,salon_3,0.33
